In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import sys

# Preliminary method 
To obtain coordinates and bonds from the database, you will need this method which trasform a matrix described as a string in dataframe cell to a `numpy` array.

In [20]:
def string_toarray(dataframe,position1,position2,element):
    if element == "coordinates":
        array = dataframe.iloc[position1][position2]
        array = array.replace("[","")
        array = array.replace("]","")
        array = array.split(',')
        array = np.array(array)
        array = array.astype(float)
        dimensions = 3
        points = int(np.size(array)/dimensions)
        array = np.reshape(array,(points,dimensions))
    
        return array
    if element == "bonds":
        array = dataframe.iloc[position1][position2]
        array = array.replace("[","")
        array = array.replace("]","")
        array = array.split(',')
        array = np.array(array)
        array = array.astype(float)
        dimensions = 2
        points = int(np.size(array)/dimensions)
        array = np.reshape(array,(points,dimensions))
        array = array.astype(int)-1
        
        return array

This method has as input the dataframe obtained by the catalogue, the position of the cell using `position1` and `position2`, and of course you have to say if you are dealing with coordinates or bonds.

# Defining a class to analyze frames of database 
We will observe all the methods which compose this class, starting with the creation of the object `Frame` and enriching this class to obtain a complete analysis of the object.

## Coordinates and bonds
The defining variables of an object `Frame` are coordinates and bonds. Please note that a `Frame` is composed by $N$ nodes and $N_c$ bonds, so the variable `coordinates` will be a $N \times 3$ matrix because we have for every node 3 coordinates. The variable `bonds` will be a $N_c \times 2$ matrix, indeed every bond is a link between 2 nodes.

In [56]:
class Frame:
    def __init__(self,coordinates,bonds,name):
        self.name = name #Name of the lattices
        self.coordinates = coordinates #Coordinates matrix
        self.bonds = bonds #Bond matrix
        self.n = n #Vector of scaling exponents (nx,ny,nz)
    

## Plotting a frame
We add to the class the method `plotting` to have a raw 3D plot of the cell, however is suggested to use the external software Ovito to plot the frames. Note that every unit cell is enclosed in $1 \times 1 \times 1$ cube.

In [57]:
class Frame:
    
    def __init__(self,coordinates,bonds,name):
        self.name = name
        self.coordinates = coordinates
        self.bonds = bonds
        self.n = n
        
    def plotting(self):
        plt.rcParams["figure.figsize"] = [7.50, 3.50]
        plt.rcParams["figure.autolayout"] = True
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")
        ax.scatter(self.coordinates[:,0],self.coordinates[:,1],self.coordinates[:,2], c='red', s=100)

        x= np.zeros((np.size(self.bonds[:,0]),2))
        y= np.zeros((np.size(self.bonds[:,0]),2))
        z= np.zeros((np.size(self.bonds[:,0]),2))

        for i in range(np.size(self.bonds[:,0])):
            x[i,0] = self.coordinates[self.bonds[i,0],0]
            x[i,1] = self.coordinates[self.bonds[i,1],0]
            y[i,0] = self.coordinates[self.bonds[i,0],1]
            y[i,1] = self.coordinates[self.bonds[i,1],1]
            z[i,0] = self.coordinates[self.bonds[i,0],2]
            z[i,1] = self.coordinates[self.bonds[i,1],2]

        for i in range(np.size(self.bonds[:,0])):
            ax.plot(x[i,:] , y[i,:], z[i,:], color='black')

        plt.show()
        

## Adjacency Matrix
Every frame can be seen as a network so we can obtain the adjacency matrix. Every matrix element will have a unity for every bonds between two nodes.

In [58]:
class Frame:
    
    def __init__(self,coordinates,bonds,name):
        self.name = name
        self.coordinates = coordinates
        self.bonds = bonds
        self.n = n
        self.periodic_bonds = [] #You can add periodic bonds matrix if you're dealing with PBC
        self.AM = [] #New variable adjacency matrix
        self.AM_P = [] #Adjacency matrix with PBC
        
    def plotting(self):
        plt.rcParams["figure.figsize"] = [7.50, 3.50]
        plt.rcParams["figure.autolayout"] = True
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")
        ax.scatter(self.coordinates[:,0],self.coordinates[:,1],self.coordinates[:,2], c='red', s=100)

        x= np.zeros((np.size(self.bonds[:,0]),2))
        y= np.zeros((np.size(self.bonds[:,0]),2))
        z= np.zeros((np.size(self.bonds[:,0]),2))

        for i in range(np.size(self.bonds[:,0])):
            x[i,0] = self.coordinates[self.bonds[i,0],0]
            x[i,1] = self.coordinates[self.bonds[i,1],0]
            y[i,0] = self.coordinates[self.bonds[i,0],1]
            y[i,1] = self.coordinates[self.bonds[i,1],1]
            z[i,0] = self.coordinates[self.bonds[i,0],2]
            z[i,1] = self.coordinates[self.bonds[i,1],2]

        for i in range(np.size(self.bonds[:,0])):
            ax.plot(x[i,:] , y[i,:], z[i,:], color='black')

        plt.show()
    
    def adjacency_matrix(self,PBC=False):
        N = len(self.coordinates[:,0])
        adjacency_matrix = np.zeros((N,N))
    
        for i in range(len(self.bonds[:,0])):
            r = self.bonds[i,0]
            c = self.bonds[i,1]
        
            adjacency_matrix[r,c]=1
            adjacency_matrix[c,r]=1
        
        self.AM.append(adjacency_matrix)
        
        if PBC == True: #Note that if PBC is turned on True you'll need the list of periodic bonds!
            for i in range(len(self.periodic_bonds[:,0])):
                r_p = self.periodic_bonds[i,0]
                c_p = self.periodic_bonds[i,1]
                adjacency_matrix[r_p,c_p]+=1
                adjacency_matrix[c_p,r_p]+=1
            
            self.AM_P.append(adjacency_matrix)

## Hessian Matrix
We add a method to compute the Hessian matrix.

In [59]:
class Frame:
    
    def __init__(self,coordinates,bonds,name):
        self.name = name
        self.coordinates = coordinates
        self.bonds = bonds
        self.n = n
        self.periodic_bonds = [] #You can add periodic bonds matrix if you're dealing with PBC
        self.AM = [] #New variable adjacency matrix
        self.AM_P = [] #Adjacency matrix with PBC
        self.H = [] #Hessian matrix in open boundary conditions
        self.H_P = [] #Hessian matrix with PBC
    
        
    def plotting(self):
        plt.rcParams["figure.figsize"] = [7.50, 3.50]
        plt.rcParams["figure.autolayout"] = True
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")
        ax.scatter(self.coordinates[:,0],self.coordinates[:,1],self.coordinates[:,2], c='red', s=100)

        x= np.zeros((np.size(self.bonds[:,0]),2))
        y= np.zeros((np.size(self.bonds[:,0]),2))
        z= np.zeros((np.size(self.bonds[:,0]),2))

        for i in range(np.size(self.bonds[:,0])):
            x[i,0] = self.coordinates[self.bonds[i,0],0]
            x[i,1] = self.coordinates[self.bonds[i,1],0]
            y[i,0] = self.coordinates[self.bonds[i,0],1]
            y[i,1] = self.coordinates[self.bonds[i,1],1]
            z[i,0] = self.coordinates[self.bonds[i,0],2]
            z[i,1] = self.coordinates[self.bonds[i,1],2]

        for i in range(np.size(self.bonds[:,0])):
            ax.plot(x[i,:] , y[i,:], z[i,:], color='black')

        plt.show()
    
    def compute_adjacency_matrix(self,PBC=False):
        N = len(self.coordinates[:,0])
        adjacency_matrix = np.zeros((N,N))
    
        for i in range(len(self.bonds[:,0])):
            r = self.bonds[i,0]
            c = self.bonds[i,1]
        
            adjacency_matrix[r,c]=1
            adjacency_matrix[c,r]=1
        
        self.AM.append(adjacency_matrix)
        
        if PBC == True: #Note that if PBC is turned on True you'll need the list of periodic bonds!
            for i in range(len(self.periodic_bonds[:,0])):
                r_p = self.periodic_bonds[i,0]
                c_p = self.periodic_bonds[i,1]
                adjacency_matrix[r_p,c_p]+=1
                adjacency_matrix[c_p,r_p]+=1
            
            self.AM_P.append(adjacency_matrix)
    
    
    
    def __Hessian_super_element_non_diagonal(self,adjacency_matrix,i,j):
        H_nd = np.zeros((3,3))
    
        distance = np.sqrt(np.power(self.coordinates[i,0]-self.coordinates[j,0],2)+np.power(self.coordinates[i,1]-self.coordinates[j,1],2)+np.power(self.coordinates[i,2]-self.coordinates[j,2],2))
        for k in range(3):
            for l in range(3):
                H_nd[k,l] = -adjacency_matrix[i,j]*(self.coordinates[i,k]-self.coordinates[j,k])*(self.coordinates[i,l]-self.coordinates[j,l])/np.power(distance,2)
    
    #print(H_nd)
        return H_nd

    def __Hessian_super_element_diagonal(self,adjacency_matrix,i):
        H_d = np.zeros((3,3))
    
        supersum = 0 
    
        for j in range(adjacency_matrix.shape[0]):
            if j!=i:
                supersum = supersum + __Hessian_super_element_non_diagonal(self,adjacency_matrix,i,j)
    
        H_d = -supersum
        #print(H_d)
        return H_d
    
    def compute_hessian_matrix(adjacency_matrix,self):
    
        H = np.zeros((3*adjacency_matrix.shape[0],3*adjacency_matrix.shape[0]))
    
        for i in range(adjacency_matrix.shape[0]):
            for j in range(adjacency_matrix.shape[0]):
                if j!=i:
                    H[i*3:i*3+3,j*3:j*3+3]=Hessian_super_element_non_diagonal(self,adjacency_matrix,i,j)
                if j == i:
                    H[i*3:i*3+3,j*3:j*3+3]=Hessian_super_element_diagonal(self,adjacency_matrix,j)
                
        return H


## Some more useful methods
We add some other methods here listed:
+ Count the number of nodes $N$
+ Count the number of bonds $N_c$
+ Count the total number of eigenvalues of the hessian matrix $N_{TOT}$
+ Count the number of non trivial zero eigenvalues in open boundary conditions $N_{OPEN}$
+ Count the number of non trivial zero eigenvalues in periodic boundary conditions $N_{PBC}$


In [63]:
class Frame:
    
    def __init__(self,coordinates,bonds,name):
        self.name = name
        self.coordinates = coordinates
        self.bonds = bonds
        self.n = n
        self.periodic_bonds = [] #You can add periodic bonds matrix if you're dealing with PBC
        self.AM = [] #New variable adjacency matrix
        self.AM_P = [] #Adjacency matrix with PBC
        self.H = [] #Hessian matrix in open boundary conditions
        self.H_P = [] #Hessian matrix with PBC
        self.N = len(self.coordinates[:,0])
        self.Nc = len(self.bonds[:,0])
        self.NTOT = []
        self.NOPEN = []
        self.NPBC = []
        
    def plotting(self):
        plt.rcParams["figure.figsize"] = [7.50, 3.50]
        plt.rcParams["figure.autolayout"] = True
        fig = plt.figure()
        ax = fig.add_subplot(projection="3d")
        ax.scatter(self.coordinates[:,0],self.coordinates[:,1],self.coordinates[:,2], c='red', s=100)

        x= np.zeros((np.size(self.bonds[:,0]),2))
        y= np.zeros((np.size(self.bonds[:,0]),2))
        z= np.zeros((np.size(self.bonds[:,0]),2))

        for i in range(np.size(self.bonds[:,0])):
            x[i,0] = self.coordinates[self.bonds[i,0],0]
            x[i,1] = self.coordinates[self.bonds[i,1],0]
            y[i,0] = self.coordinates[self.bonds[i,0],1]
            y[i,1] = self.coordinates[self.bonds[i,1],1]
            z[i,0] = self.coordinates[self.bonds[i,0],2]
            z[i,1] = self.coordinates[self.bonds[i,1],2]

        for i in range(np.size(self.bonds[:,0])):
            ax.plot(x[i,:] , y[i,:], z[i,:], color='black')

        plt.show()
    
    def compute_adjacency_matrix(self,PBC=False):
        N = len(self.coordinates[:,0])
        adjacency_matrix = np.zeros((N,N))
    
        for i in range(len(self.bonds[:,0])):
            r = self.bonds[i,0]
            c = self.bonds[i,1]
        
            adjacency_matrix[r,c]=1
            adjacency_matrix[c,r]=1
        
        self.AM.append(adjacency_matrix)
        
        if PBC == True: #Note that if PBC is turned on True you'll need the list of periodic bonds!
            for i in range(len(self.periodic_bonds[:,0])):
                r_p = self.periodic_bonds[i,0]
                c_p = self.periodic_bonds[i,1]
                adjacency_matrix[r_p,c_p]+=1
                adjacency_matrix[c_p,r_p]+=1
            
            self.AM_P.append(adjacency_matrix)
    
    
    
    def __Hessian_super_element_non_diagonal(self,adjacency_matrix,i,j):
        H_nd = np.zeros((3,3))
    
        distance = np.sqrt(np.power(self.coordinates[i,0]-self.coordinates[j,0],2)+np.power(self.coordinates[i,1]-self.coordinates[j,1],2)+np.power(self.coordinates[i,2]-self.coordinates[j,2],2))
        for k in range(3):
            for l in range(3):
                H_nd[k,l] = -adjacency_matrix[i,j]*(self.coordinates[i,k]-self.coordinates[j,k])*(self.coordinates[i,l]-self.coordinates[j,l])/np.power(distance,2)
    
    #print(H_nd)
        return H_nd

    def __Hessian_super_element_diagonal(self,adjacency_matrix,i):
        H_d = np.zeros((3,3))
    
        supersum = 0 
    
        for j in range(adjacency_matrix.shape[0]):
            if j!=i:
                supersum = supersum + __Hessian_super_element_non_diagonal(self,adjacency_matrix,i,j)
    
        H_d = -supersum
        #print(H_d)
        return H_d
    
    def compute_hessian_matrix(adjacency_matrix,self):
    
        H = np.zeros((3*adjacency_matrix.shape[0],3*adjacency_matrix.shape[0]))
    
        for i in range(adjacency_matrix.shape[0]):
            for j in range(adjacency_matrix.shape[0]):
                if j!=i:
                    H[i*3:i*3+3,j*3:j*3+3]=Hessian_super_element_non_diagonal(self,adjacency_matrix,i,j)
                if j == i:
                    H[i*3:i*3+3,j*3:j*3+3]=Hessian_super_element_diagonal(self,adjacency_matrix,j)
                
        return H

    def compute_eigen_space(self,PBC = False):
        if PBC == False:
            Eigenvalues, Eigenvectors = np.linalg.eig(self.H)
            self.NTOT.append(len(Eigenvalues))
            k = 0
            for i in range(np.size(eigenvalues)):
                if np.abs(eigenvalues[i].real) < np.exp(-22) : 
                    k = k + 1
            self.NOPEN.append(k-6)
        if PBC == True:
            Eigenvalues, Eigenvectors = np.linalg.eig(self.H_P)
            k = 0
            for i in range(np.size(eigenvalues)):
                if np.abs(eigenvalues[i].real) < np.exp(-22) : 
                    k = k + 1
            self.NPBC.append(k-6)

In [34]:
coordinates = np.matrix([[0,0,0],[0,1,0],[1,0,0],[1,1,0],[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
bonds = np.matrix([[0,4],[0,1],[0,2],[1,5],[1,3],[2,6],[2,3],[3,7],[4,5],[4,6],[5,7],[6,7]])

In [35]:
cubic = Frame(coordinates,bonds)

In [ ]:
cubic.plotting()

# Expansion of a lattice
Now we introduce some external functions that complete the analysis on the frame. This group of functions carries out the expansion of the frame in any direction using as input the $S$ parameter which represents the expansion. For the unit cell enclosed in a $1 \times 1 \times 1$ cube $S=0$.  

In [52]:
#shape is S variabile 

def distance_from_center(coordinates,center):
    distances = np.zeros((len(coordinates[:,0]),3))
    for i in range(len(coordinates[:,0])):
        for k in range(3):
            distances[i][k] = coordinates[i][k]-center[k]
    return distances



def expand_coordinates(shape,coordinates,center,traslation):
    distances = distance_from_center(coordinates,center)
    new_coordinate = np.zeros(3)
    new_center = np.zeros(3)
    for i in range(shape[0]+1):
        for k in range(shape[1]+1):
            for m in range(shape[2]+1):
                new_center[0] = center[0] + traslation*i
                new_center[1] = center[1] + traslation*k
                new_center[2] = center[2] + traslation*m
                
                for j in range(len(distances[:,0])):
                    new_coordinate[0] = new_center[0]+distances[j][0]
                    new_coordinate[1] = new_center[1]+distances[j][1]
                    new_coordinate[2] = new_center[2]+distances[j][2]
                    flag = False
                    for s in range(len(coordinates[:,0])):
                        if np.abs(coordinates[s,0] - new_coordinate[0])<np.exp(-22)  and np.abs(coordinates[s,1]-new_coordinate[1])<np.exp(-22) and np.abs(coordinates[s,2] - new_coordinate[2])<np.exp(-22) : flag = True
                    
                    if flag==False: coordinates = np.append(coordinates,[new_coordinate],axis=0)
                """
                if flag == True: 
                    print("esiste già")
                    print(new_coordinate)
                    print(" ")
                """
    """
    for l in range(len(coordinates[:,0])):
        count = 0
        for n in range(len(coordinates[:,0])):
            if coordinates[n,0] == coordinates[l,0] and coordinates[n,1] == coordinates[l,1]: count = count +1
        if count>1 : coordinates = np.delete(coordinates,(n),axis=0)  
    """
    return coordinates


def expand_bonds(shape,coordinates,bonds,traslation):
    coordinate1 = np.zeros(3)
    coordinate2 = np.zeros(3)
    new_coordinate1 = np.zeros(3)
    new_coordinate2 = np.zeros(3)
    new_bond = np.zeros(2)
    new_bonds = bonds
    for s in range(len(bonds[:,0])):
        coordinate1[0] = coordinates[bonds[s,0],0]
        coordinate1[1] = coordinates[bonds[s,0],1]
        coordinate1[2] = coordinates[bonds[s,0],2]
        coordinate2[0] = coordinates[bonds[s,1],0]
        coordinate2[1] = coordinates[bonds[s,1],1]
        coordinate2[2] = coordinates[bonds[s,1],2]
        for i in range(shape[0]+1):
            for k in range(shape[1]+1):
                for j in range(shape[2]+1):
                    new_coordinate1[0] = coordinate1[0] + traslation*i
                    new_coordinate1[1] = coordinate1[1] + traslation*k
                    new_coordinate1[2] = coordinate1[2] + traslation*j
                    new_coordinate2[0] = coordinate2[0] + traslation*i
                    new_coordinate2[1] = coordinate2[1] + traslation*k
                    new_coordinate2[2] = coordinate2[2] + traslation*j
                    #print(new_coordinate1,new_coordinate2)
              
                
            
                    for l in range(len(coordinates[:,0])):
                        if np.abs(coordinates[l,0]-new_coordinate1[0])<np.exp(-22) and np.abs(coordinates[l,1]-new_coordinate1[1])<np.exp(-22) and np.abs(coordinates[l,2]-new_coordinate1[2])<np.exp(-22): new_bond[0]=l

                    for h in range(len(coordinates[:,0])):
                        if np.abs(coordinates[h,0]-new_coordinate2[0])<np.exp(-22) and np.abs(coordinates[h,1]-new_coordinate2[1])<np.exp(-22) and np.abs(coordinates[h,2]-new_coordinate2[2])<np.exp(-22): new_bond[1]=h
                
                    #print(new_bond)
                    flag = False
                
                    for v in range(len(new_bonds[:,0])):
                        if np.abs(new_bonds[v,0]-new_bond[0])<np.exp(-22) and np.abs(new_bonds[v,1]-new_bond[1])<np.exp(-22) : flag = True
                        if np.abs(new_bonds[v,1]-new_bond[0])<np.exp(-22) and np.abs(new_bonds[v,0]-new_bond[1])<np.exp(-22) : flag = True
                        if np.abs(new_bond[0]-new_bond[1])<np.exp(-22) : flag = True
                    
                    if flag==False: new_bonds = np.append(new_bonds,[new_bond],axis=0)
                
    new_bonds=new_bonds.astype(int)           
    return new_bonds



def expand_lattice(coordinates,bonds,shape):
    center = np.array([0.5,0.5,0.5]) #center of a unit cell enclosed in a 1x1x1 cube
    traslation = 1 #default
    coordinates_expanded = expand_coordinates(shape,coordinates,center,traslation)
    bonds_expanded = expand_bonds(shape,new_coordinates,bonds,traslation)
    return coordinates_expanded,bonds_expanded

# PBC on a lattice
This method is useful to obtain the list of periodic bonds from a lattice applying PBC. Note that due to some frames that has degenerations is recommended to expand lattices at least with $S=1$ before applying PBC.

In [53]:
def periodic_cell(coordinates,bonds):
    
    new_coordinates = coordinates
    periodic_bonds = bonds
    nonzeropoints=np.zeros(1)
    bonds_trasformationx=np.zeros((1,2))
    bonds_trasformationy=np.zeros((1,2))
    bonds_trasformationz=np.zeros((1,2))
    
    
    #Seleziono le facce iniziali ossia quelle con coordinata nulla
    
    for i in range(len(coordinates[:,0])):
        if coordinates[i,0]!=0 and coordinates[i,1]!=0 and coordinates[i,2]!=0: nonzeropoints = np.append(nonzeropoints,i)
    
    nonzeropoints = np.delete(nonzeropoints,0)
    nonzeropoints = nonzeropoints.astype(int)
    nonzeropoints = list(set(nonzeropoints))
    new_coordinates = np.delete(new_coordinates,nonzeropoints,axis=0)
    
    #Trovo l'omologo sulle facce nei tre casi
    
    for i in range(len(new_coordinates[:,0])):
        for k in range(len(coordinates[:,0])):
            if coordinates[k,0] == new_coordinates[i,0] and coordinates[k,1] == new_coordinates[i,1] and coordinates[k,2] == new_coordinates[i,2]: point_number0 = k
        
        
        #CASO 3 COORDINATE NULLE
        if new_coordinates[i,0] <np.exp(-22) and new_coordinates[i,1] <np.exp(-22) and new_coordinates[i,2] <np.exp(-22):
            coordinate_omologuex =[2,0,0]
            coordinate_omologuey =[0,2,0]
            coordinate_omologuez =[0,0,2]
            
            Flagx = False
            Flagy = False
            Flagz = False
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuex[0] and coordinates[j,1] == coordinate_omologuex[1] and coordinates[j,2] == coordinate_omologuex[2]:
                    point_numberx = j
                    Flagx = True
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuey[0] and coordinates[j,1] == coordinate_omologuey[1] and coordinates[j,2] == coordinate_omologuey[2]:
                    point_numbery = j
                    Flagy = True
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuez[0] and coordinates[j,1] == coordinate_omologuez[1] and coordinates[j,2] == coordinate_omologuez[2]:
                    point_numberz = j
                    Flagz = True
            
            if Flagx == True:
                bonds_trasformationx = np.append(bonds_trasformationx,[[point_number0,point_numberx]],axis=0)
            
            if Flagy == True:
                bonds_trasformationy = np.append(bonds_trasformationy,[[point_number0,point_numbery]],axis=0)
            
            if Flagz == True:
                bonds_trasformationz = np.append(bonds_trasformationz,[[point_number0,point_numberz]],axis=0)
            
            
        #CASI 2 COORDINATE NULLE
        
        elif new_coordinates[i,0] <np.exp(-22) and new_coordinates[i,1] <np.exp(-22):
            coordinate_omologuex =[2,0,new_coordinates[i,2]]
            coordinate_omologuey =[0,2,new_coordinates[i,2]]
            
            Flagx = False
            Flagy = False
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuex[0] and coordinates[j,1] == coordinate_omologuex[1] and coordinates[j,2] == coordinate_omologuex[2]:
                    point_numberx = j
                    Flagx = True
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuey[0] and coordinates[j,1] == coordinate_omologuey[1] and coordinates[j,2] == coordinate_omologuey[2]:
                    point_numbery = j
                    Flagy = True
            
            if Flagx == True:
                bonds_trasformationx = np.append(bonds_trasformationx,[[point_number0,point_numberx]],axis=0)
            
            if Flagy == True:
                bonds_trasformationy = np.append(bonds_trasformationy,[[point_number0,point_numbery]],axis=0)
        
        
        elif new_coordinates[i,1] <np.exp(-22) and new_coordinates[i,2] <np.exp(-22):
            coordinate_omologuez =[new_coordinates[i,0],0,2]
            coordinate_omologuey =[new_coordinates[i,0],2,0]
            
            Flagz = False
            Flagy = False
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuez[0] and coordinates[j,1] == coordinate_omologuez[1] and coordinates[j,2] == coordinate_omologuez[2]:
                    point_numberz = j
                    Flagz = True
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuey[0] and coordinates[j,1] == coordinate_omologuey[1] and coordinates[j,2] == coordinate_omologuey[2]:
                    point_numbery = j
                    Flagy = True
            
            if Flagz == True:
                bonds_trasformationz = np.append(bonds_trasformationz,[[point_number0,point_numberz]],axis=0)
            
            if Flagy == True:
                bonds_trasformationy = np.append(bonds_trasformationy,[[point_number0,point_numbery]],axis=0)
            
            
        elif new_coordinates[i,0] <np.exp(-22) and new_coordinates[i,2] <np.exp(-22):
            coordinate_omologuex =[2,new_coordinates[i,1],0]
            coordinate_omologuez =[0,new_coordinates[i,1],2]
            
            Flagx = False
            Flagz = False
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuex[0] and coordinates[j,1] == coordinate_omologuex[1] and coordinates[j,2] == coordinate_omologuex[2]:
                    point_numberx = j
                    Flagx = True
            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuez[0] and coordinates[j,1] == coordinate_omologuez[1] and coordinates[j,2] == coordinate_omologuez[2]:
                    point_numberz = j
                    Flagz = True
            
            if Flagx == True:
                bonds_trasformationx = np.append(bonds_trasformationx,[[point_number0,point_numberx]],axis=0)
                
            if Flagz == True:
                bonds_trasformationz = np.append(bonds_trasformationz,[[point_number0,point_numberz]],axis=0)
            
        #CASI AD UNA COORDINATA NULLA
        
        elif new_coordinates[i,0] <np.exp(-22):
            coordinate_omologuex =[2,new_coordinates[i,1],new_coordinates[i,2]]

            Flagx = False

            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuex[0] and coordinates[j,1] == coordinate_omologuex[1] and coordinates[j,2] == coordinate_omologuex[2]:
                    point_numberx = j
                    Flagx = True
            
            if Flagx == True:
                bonds_trasformationx = np.append(bonds_trasformationx,[[point_number0,point_numberx]],axis=0)
        
        elif new_coordinates[i,1] <np.exp(-22):
            coordinate_omologuey =[new_coordinates[i,0],2,new_coordinates[i,2]]

            Flagy = False

            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuey[0] and coordinates[j,1] == coordinate_omologuey[1] and coordinates[j,2] == coordinate_omologuey[2]:
                    point_numbery = j
                    Flagy = True
            
            if Flagy == True:
                bonds_trasformationy = np.append(bonds_trasformationy,[[point_number0,point_numbery]],axis=0)
        
        
        elif new_coordinates[i,2] <np.exp(-22):
            coordinate_omologuez =[new_coordinates[i,0],new_coordinates[i,1],2]

            Flagz = False

            
            for j in range(len(coordinates[:,0])):
                if coordinates[j,0] == coordinate_omologuez[0] and coordinates[j,1] == coordinate_omologuez[1] and coordinates[j,2] == coordinate_omologuez[2]:
                    point_numberz = j
                    Flagz = True
            
            if Flagz == True:
                bonds_trasformationz = np.append(bonds_trasformationz,[[point_number0,point_numberz]],axis=0)
    """
    print("Bonds prima")            
    print(bonds)
    """
    #INIZIO LE TRASFORMAZIONI

    bonds_trasformationx = np.delete(bonds_trasformationx,0,axis=0)
    bonds_trasformationy = np.delete(bonds_trasformationy,0,axis=0)
    bonds_trasformationz = np.delete(bonds_trasformationz,0,axis=0)
    """
    print(bonds_trasformationx)
    print(bonds_trasformationy)
    print(bonds_trasformationz)
    """
    bonds_real = np.zeros((len(bonds[:,0]),2))
    for i in range(len(bonds[:,0])):
        bonds_real[i,1] = bonds[i,1]
        bonds_real[i,0] = bonds[i,0]
    
    #TRASFORMAZIONE LUNGO X
    
    for i in range(len(bonds_trasformationx[:,0])):
        for s in range(len(periodic_bonds[:,0])):
            if np.abs(periodic_bonds[s,0]-bonds_trasformationx[i,1])<np.exp(-22):
                periodic_bonds[s,0]= bonds_trasformationx[i,0]
                
            elif np.abs(periodic_bonds[s,1]-bonds_trasformationx[i,1])<np.exp(-22):
                periodic_bonds[s,1]= bonds_trasformationx[i,0]
    
    #TRASFORMAZIONE LUNGO Y
    
    for i in range(len(bonds_trasformationy[:,0])):
        for s in range(len(periodic_bonds[:,0])):
            if np.abs(periodic_bonds[s,0]-bonds_trasformationy[i,1])<np.exp(-22):
                periodic_bonds[s,0]= bonds_trasformationy[i,0]
                
            elif np.abs(periodic_bonds[s,1]-bonds_trasformationy[i,1])<np.exp(-22):
                periodic_bonds[s,1]= bonds_trasformationy[i,0]
    
    #TRASFORMAZIONE LUNGO Z
    
    for i in range(len(bonds_trasformationz[:,0])):
        for s in range(len(periodic_bonds[:,0])):
            if np.abs(periodic_bonds[s,0]-bonds_trasformationz[i,1])<np.exp(-22):
                periodic_bonds[s,0]= bonds_trasformationz[i,0]
                
            elif np.abs(periodic_bonds[s,1]-bonds_trasformationz[i,1])<np.exp(-22):
                periodic_bonds[s,1]= bonds_trasformationz[i,0]
    
    """
    print("bonds real")
    print(bonds_real)
    print(periodic_bonds)
    
    """
    #TOLGO I BONDS NON COINVOLTI NELLA TRASFORMAZIONE
    
    
    new_periodic_bonds_indices = np.array(0)
    
    for i in range(len(periodic_bonds[:,0])):
        if np.abs(periodic_bonds[i,0]-bonds_real[i,0])<np.exp(-22) and np.abs(periodic_bonds[i,1]-bonds_real[i,1])<np.exp(-22): 
            new_periodic_bonds_indices = np.append(new_periodic_bonds_indices,i)
            
    new_periodic_bonds_indices = np.delete(new_periodic_bonds_indices,0)
    new_periodic_bonds_indices = list(set(new_periodic_bonds_indices))
    periodic_bonds = np.delete(periodic_bonds,new_periodic_bonds_indices,axis=0)
   

    
    #VERIFICHIAMO CHE NON CI SIANO BONDS RIPETUTI
    
    periodic_bonds_temp = np.array(0)
    for i in range(int(len(periodic_bonds[:,0])-1)):
        for m in range(i+1,len(periodic_bonds[:,0])):
            if periodic_bonds[m,0] == periodic_bonds[i,0] and periodic_bonds[m,1] == periodic_bonds[i,1] or periodic_bonds[m,0] == periodic_bonds[i,1] and periodic_bonds[m,1] == periodic_bonds[i,0]: periodic_bonds_temp = np.append(periodic_bonds_temp,m)
    
    
    periodic_bonds_temp = np.delete(periodic_bonds_temp,0)
    periodic_bonds_temp = list(set(periodic_bonds_temp))
    periodic_bonds = np.delete(periodic_bonds,periodic_bonds_temp,axis=0)
    
    
    return periodic_bonds, bonds_real.astype(int)